In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from glob import glob
from dateutil.relativedelta import relativedelta,TH
import warnings
warnings.filterwarnings("ignore")

In [3]:
path = pd.DataFrame(glob('C:\August-2022 Data/*'), columns = ['Location'])

In [4]:
path['datetime'] = path['Location'].apply(lambda x: x.split("_")[-1])
path['datetime'] = pd.to_datetime(path['datetime'])

In [5]:
path

,Location,datetime
0,C:\August-2022 Data\Option_data_01-08-2022,2022-01-08
1,C:\August-2022 Data\Option_data_02-08-2022,2022-02-08
2,C:\August-2022 Data\Option_data_03-08-2022,2022-03-08
3,C:\August-2022 Data\Option_data_04-08-2022,2022-04-08
4,C:\August-2022 Data\Option_data_05-08-2022,2022-05-08
5,C:\August-2022 Data\Option_data_08-08-2022,2022-08-08
6,C:\August-2022 Data\Option_data_10-08-2022,2022-10-08
7,C:\August-2022 Data\Option_data_11-08-2022,2022-11-08
8,C:\August-2022 Data\Option_data_12-08-2022,2022-12-08
9,C:\August-2022 Data\Option_data_16-08-2022,2022-08-16


In [6]:
trade_log_ce = pd.DataFrame(columns = ['Entry_date_time' , 'Future_price' , 'ATM','Day_to_exp' , 'Symbol', 'Leg1_strike' , 'Leg2_strike' , 'Leg3_strike' , 'Leg1_price' ,
             'Leg2_price' ,'Leg3_price', 'Entry_spred' , 'Stoploss' , 'Exit_spread' , 'Exit_time','pnl'])

In [7]:
trade_log_pe = pd.DataFrame(columns = ['Entry_date_time' , 'Future_price' , 'ATM','Day_to_exp' , 'Symbol', 'Leg1_strike' , 'Leg2_strike' , 'Leg3_strike' , 'Leg1_price' ,
             'Leg2_price' ,'Leg3_price', 'Entry_spread_pe' , 'Stoploss_pe' , 'Exit_spread' , 'Exit_time','pnl_pe'])

In [8]:
for index , row in path.iterrows():
    
    try:
        
       # Read data
        data= pd.read_csv(row['Location'])
        print(index)

        #create entry time and exit time
        entry_date_time=datetime.datetime.combine(row['datetime'].date(),datetime.time(9,20))
        exit_date_time=datetime.datetime.combine(row['datetime'].date(),datetime.time(15,10))

        data['Date/Time'] = pd.to_datetime(data['Date/Time'])

        data['Exp_Date'] = pd.to_datetime(data['Exp_Date'])

        # find future price
        Future_price = data[(data['Instrument'] == 'FUTIDX') & (data['Symbol'] == 'NIFTY') & (data['Date/Time'] == entry_date_time)]['Close'].iloc[0]
        # Atm Price
        base_price = 100
        atm = base_price*round(Future_price/base_price)

        # Create Frist leg , second leg and third leg strike and strike price 
        leg1_ce_strike = atm - 200
        leg2_ce_strike = leg1_ce_strike + 100
        leg3_ce_strike = leg2_ce_strike + 100

        leg1_pe_strike = atm - 100
        leg2_pe_strike = leg1_pe_strike - 200
        leg3_pe_strike = leg2_pe_strike - 200

        # Ce data
        ce_data = data[(data['Instrument'] == "OPTIDX") & (data['Option_Type'] == 'CE') & (data['Symbol'] == 'NIFTY')].reset_index(drop = 'Frist')

        # PE data
        pe_data = data[(data['Instrument'] == "OPTIDX") & (data['Option_Type'] == 'PE') & (data['Symbol'] == 'NIFTY')].reset_index(drop = 'Frist')

        Leg1_price = ce_data[(ce_data['Strike_price'] == leg1_ce_strike) & (ce_data['Date/Time'] == entry_date_time)]['Close'].iloc[0]
        Leg2_price = ce_data[(ce_data['Strike_price'] == leg2_ce_strike) & (ce_data['Date/Time'] == entry_date_time)]['Close'].iloc[0]
        Leg3_price = ce_data[(ce_data['Strike_price'] == leg3_ce_strike) & (ce_data['Date/Time'] == entry_date_time)]['Close'].iloc[0]

        Leg1_pe_price = pe_data[(pe_data['Strike_price'] == leg1_pe_strike) & (pe_data['Date/Time'] == entry_date_time)]['Close'].iloc[0]
        Leg2_pe_price = pe_data[(pe_data['Strike_price'] == leg2_pe_strike) & (pe_data['Date/Time'] == entry_date_time)]['Close'].iloc[0]
        Leg3_pe_price = pe_data[(pe_data['Strike_price'] == leg3_pe_strike) & (pe_data['Date/Time'] == entry_date_time)]['Close'].iloc[0]

        ce_symbol = ce_data['Option_Type'].iloc[0]
        pe_symbol = pe_data['Option_Type'].iloc[0]

        # create Spread
        entry_spread_ce = round((Leg1_price - 2*(Leg2_price) + Leg3_price),2)

        entry_spread_pe =  round((Leg1_pe_price - 2*(Leg2_pe_price) + Leg3_pe_price),2) 

        # create Stoploss
        stoploss = (entry_spread_ce *80)/100

        stoploss_pe = (entry_spread_pe *80)/100


        data['Timestamp'] = pd.to_datetime(data['Timestamp'],format='%d-%m-%Y')

        Day_to_exp = (data['Exp_Date'].dt.day) - (data['Timestamp'].dt.day)

        Day_to_exp = Day_to_exp[0]

        future_data = data[(data['Instrument'] == 'FUTIDX') & (data['Symbol'] == 'NIFTY')].reset_index(drop='Frist')
        future_data = future_data[['Date/Time' , 'Close']].set_index(['Date/Time'])


        leg1_price = ce_data[ce_data['Strike_price'] == leg1_ce_strike][['Date/Time','Close']].reset_index(drop='Frist')
        leg1_price=leg1_price.set_index(['Date/Time'])

        leg2_price = ce_data[ce_data['Strike_price'] == leg2_ce_strike][['Date/Time','Close']].reset_index(drop='Frist')
        leg2_price=leg2_price.set_index(['Date/Time'])

        leg3_price = ce_data[ce_data['Strike_price'] == leg3_ce_strike][['Date/Time','Close']].reset_index(drop='Frist')
        leg3_price=leg3_price.set_index(['Date/Time'])

        #create intrday Data for ce 
        intrday_data = leg1_price-2*(leg2_price)+leg3_price

        intrday_data['Symbol'] ="CE"

        intrday_data['pnl'] = 0

        intrday_data.rename({"Close" : 'Close_Spread'} , axis =1 , inplace =True)

        intrday_data = intrday_data.reset_index()

        leg1_pe_price = pe_data[pe_data['Strike_price'] == leg1_pe_strike][['Date/Time','Close']].reset_index(drop='Frist')
        leg1_pe_price=leg1_pe_price.set_index(['Date/Time'])

        leg2_pe_price = pe_data[pe_data['Strike_price'] == leg2_pe_strike][['Date/Time','Close']].reset_index(drop='Frist')
        leg2_pe_price=leg2_pe_price.set_index(['Date/Time'])

        leg3_pe_price = pe_data[pe_data['Strike_price'] == leg3_pe_strike][['Date/Time','Close']].reset_index(drop='Frist')
        leg3_pe_price=leg3_pe_price.set_index(['Date/Time'])

        # Create intrday data for put
        intrday_data1 = leg1_pe_price-2*(leg2_pe_price)+leg3_pe_price

        intrday_data1['Symbol'] ="PE"

        intrday_data1['pnl_pe'] = 0

        intrday_data1 = intrday_data1.reset_index()

        intrday_data1.rename({"Close" : 'Close_Spread'} , axis =1 , inplace =True)

        intrday_data = intrday_data.reset_index(drop = "Frist")

        # Backtesting

        ce_stop_loss_counter =0
        ce_exit_datetime = ''
        ce_spread_exit_price = 0
        pnl =0


        for index,row in intrday_data.iterrows():
            ce_ltp = row['Close_Spread']

            # Not hit stoploss and exit 15:10 condition
            if (ce_stop_loss_counter == 0) & (row['Date/Time'] == exit_date_time):
                pnl = ce_ltp - entry_spread_ce
                ce_stop_loss_counter = 1
                ce_exit_datetime = row['Date/Time']
                ce_spread_exit_price = ce_ltp
                #indrday_data['pnl'] = pnl
                print('Not Hit Stop loss Position Exit 15:10')
                break

            #stoploss hit condtition
            elif ce_ltp<=stoploss:
                pnl = ce_ltp - entry_spread_ce
                ce_stop_loss_counter = 0
                ce_exit_datetime = row['Date/Time']
                ce_spread_exit_price = ce_ltp
                intrday_data.loc[index,'pnl'] = pnl
                print('Stop Loss Hit')
                break

        trade_log_ce= trade_log_ce.append({'Entry_date_time': entry_date_time , 'Future_price' : Future_price , 'ATM':atm,'Day_to_exp' : Day_to_exp , 'Symbol' : ce_symbol, 'Leg1_strike':leg1_ce_strike, 'Leg2_strike':leg2_ce_strike , 'Leg3_strike':leg3_ce_strike , 'Leg1_price':Leg1_price ,
                 'Leg2_price':Leg2_price ,'Leg3_price':Leg3_price, 'Entry_spred':entry_spread_ce , 'Stoploss':stoploss , 'Exit_spread':ce_spread_exit_price ,'Exit_time': ce_exit_datetime, 'pnl':pnl},ignore_index =True)

        #('-------------------------------------------------PE Backtesting----------------------------------------')

        pe_stop_loss_counter =0
        pe_exit_datetime =''
        pe_spread_exit_price = 0
        pnl_pe =0

        for index_1 , row_1 in intrday_data1.iterrows():
            pe_ltp = row_1['Close_Spread'] 
             # Not hit stoploss and exit 15:10
            if (pe_stop_loss_counter == 0) & (row_1['Date/Time'] == exit_date_time):
                pnl_pe = pe_ltp - entry_spread_pe
                pe_stop_loss_counter = 1
                pe_exit_datetime = row_1['Date/Time']
                pe_spread_exit_price = pe_ltp
                intrday_data1.loc[index,'pnl'] = pnl_pe
                print('Not Hit Stop loss Position Exit 15:10')
                break
                # stoploss hit condtition
            elif pe_ltp<=stoploss_pe:
                pnl_pe = pe_ltp - entry_spread_pe
                pe_stop_loss_counter = 0
                pe_exit_datetime = row_1['Date/Time']
                pe_spread_exit_price = pe_ltp
                intrday_data1.loc[index,'pnl_pe'] = pnl_pe
                print('Stop Loss Hit')
                break 
        trade_log_pe = trade_log_pe.append({'Entry_date_time': entry_date_time , 'Future_price' : Future_price , 'ATM':atm,'Day_to_exp' : Day_to_exp , 'Symbol' : pe_symbol, 'Leg1_strike':leg1_pe_strike, 'Leg2_strike':leg2_pe_strike , 'Leg3_strike':leg3_pe_strike , 'Leg1_price':Leg1_pe_price ,
   
                                        'Leg2_price':Leg2_pe_price ,'Leg3_price':Leg3_pe_price, 'Entry_spread_pe':entry_spread_pe , 'Stoploss_pe':stoploss_pe , 'Exit_spread':pe_spread_exit_price ,'Exit_time': pe_exit_datetime, 'pnl_pe' : pnl_pe},ignore_index =True)
    except Exception as e:
        print(e)
        print(row['Location'])
        

0
Stop Loss Hit
Stop Loss Hit
1
Stop Loss Hit
Stop Loss Hit
2
Stop Loss Hit
Stop Loss Hit
3
Stop Loss Hit
Not Hit Stop loss Position Exit 15:10
4
Stop Loss Hit
Stop Loss Hit
5
Stop Loss Hit
Stop Loss Hit
6
Stop Loss Hit
Not Hit Stop loss Position Exit 15:10
7
Stop Loss Hit
Not Hit Stop loss Position Exit 15:10
8
Stop Loss Hit
Not Hit Stop loss Position Exit 15:10
9
Stop Loss Hit
Not Hit Stop loss Position Exit 15:10
10
Stop Loss Hit
Stop Loss Hit
11
Not Hit Stop loss Position Exit 15:10
Not Hit Stop loss Position Exit 15:10
12
Not Hit Stop loss Position Exit 15:10
Not Hit Stop loss Position Exit 15:10
13
Not Hit Stop loss Position Exit 15:10
Not Hit Stop loss Position Exit 15:10
14
Stop Loss Hit
Stop Loss Hit
15
Not Hit Stop loss Position Exit 15:10
Stop Loss Hit
16
Stop Loss Hit
Stop Loss Hit


In [9]:
trade_log_ce

,Entry_date_time,Future_price,ATM,Day_to_exp,Symbol,Leg1_strike,Leg2_strike,Leg3_strike,Leg1_price,Leg2_price,Leg3_price,Entry_spred,Stoploss,Exit_spread,Exit_time,pnl
0,2022-01-08 09:20:00,17192.7,17200,24,CE,17000,17100,17200,398.45,338.00,282.75,5.20,4.16,3.90,2022-01-08 09:18:00,-1.30
1,2022-02-08 09:20:00,17319.8,17300,23,CE,17100,17200,17300,415.25,352.20,294.00,4.85,3.88,3.60,2022-02-08 09:15:00,-1.25
2,2022-03-08 09:20:00,17395.0,17400,22,CE,17200,17300,17400,409.35,346.65,288.40,4.45,3.56,3.35,2022-03-08 09:19:00,-1.10
3,2022-04-08 09:20:00,17462.5,17500,21,CE,17300,17400,17500,374.20,315.15,259.00,2.90,2.32,2.05,2022-04-08 09:17:00,-0.85
4,2022-05-08 09:20:00,17439.4,17400,20,CE,17200,17300,17400,425.00,358.85,300.20,7.50,6.00,5.50,2022-05-08 09:23:00,-2.00
5,2022-08-08 09:20:00,17399.4,17400,17,CE,17200,17300,17400,383.15,317.75,258.70,6.35,5.08,1.20,2022-08-08 09:15:00,-5.15
6,2022-10-08 09:20:00,17548.8,17500,15,CE,17300,17400,17500,408.00,339.45,279.40,8.50,6.80,5.40,2022-10-08 09:15:00,-3.10
7,2022-11-08 09:20:00,17718.1,17700,14,CE,17500,17600,17700,352.00,286.30,227.05,6.45,5.16,5.10,2022-11-08 09:16:00,-1.35
8,2022-12-08 09:20:00,17659.0,17700,13,CE,17500,17600,17700,295.95,232.75,176.65,7.10,5.68,5.15,2022-12-08 09:17:00,-1.95
9,2022-08-16 09:20:00,17795.0,17800,9,CE,17600,17700,17800,294.45,226.20,168.20,10.25,8.20,8.10,2022-08-16 09:29:00,-2.15


In [10]:
trade_log_pe

,Entry_date_time,Future_price,ATM,Day_to_exp,Symbol,Leg1_strike,Leg2_strike,Leg3_strike,Leg1_price,Leg2_price,Leg3_price,Entry_spread_pe,Stoploss_pe,Exit_spread,Exit_time,pnl_pe
0,2022-01-08 09:20:00,17192.7,17200,24,PE,17100,16900,16700,244.50,173.65,122.45,19.65,15.72,15.70,2022-01-08 10:57:00,-3.95
1,2022-02-08 09:20:00,17319.8,17300,23,PE,17200,17000,16800,236.60,169.00,120.20,18.80,15.04,14.45,2022-02-08 14:34:00,-4.35
2,2022-03-08 09:20:00,17395.0,17400,22,PE,17300,17100,16900,256.15,186.15,134.90,18.75,15.00,13.65,2022-03-08 09:56:00,-5.10
3,2022-04-08 09:20:00,17462.5,17500,21,PE,17400,17200,17000,254.40,182.85,130.00,18.70,14.96,21.00,2022-04-08 15:10:00,2.30
4,2022-05-08 09:20:00,17439.4,17400,20,PE,17300,17100,16900,219.40,155.90,109.80,17.40,13.92,13.40,2022-05-08 09:48:00,-4.00
5,2022-08-08 09:20:00,17399.4,17400,17,PE,17300,17100,16900,219.30,152.50,104.85,19.15,15.32,15.25,2022-08-08 10:02:00,-3.90
6,2022-10-08 09:20:00,17548.8,17500,15,PE,17400,17200,17000,191.05,131.35,89.25,17.60,14.08,18.95,2022-10-08 15:10:00,1.35
7,2022-11-08 09:20:00,17718.1,17700,14,PE,17600,17400,17200,168.75,108.00,69.40,22.15,17.72,23.60,2022-11-08 15:10:00,1.45
8,2022-12-08 09:20:00,17659.0,17700,13,PE,17600,17400,17200,174.65,108.10,65.75,24.20,19.36,24.50,2022-12-08 15:10:00,0.30
9,2022-08-16 09:20:00,17795.0,17800,9,PE,17700,17500,17300,131.60,75.10,42.75,24.15,19.32,21.20,2022-08-16 15:10:00,-2.95
